In [1]:
# adapted based on https://www.tensorflow.org/tutorials/keras/regression
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2024-02-27 23:45:23.175823: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 23:45:23.219523: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 23:45:23.219555: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 23:45:23.219579: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 23:45:23.227849: I tensorflow/core/platform/cpu_feature_g

In [2]:
raw_dataset = pd.read_csv('linear-relationship.csv')
print('Done')

Done


In [3]:
dataset = raw_dataset.copy()
dataset.tail()

,x,y
9995,9995,19995
9996,9996,19997
9997,9997,19999
9998,9998,20001
9999,9999,20003


In [4]:
train_ds = dataset.sample(frac=0.8, random_state=0)
test_ds = dataset.drop(train_ds.index)
print(train_ds.tail())
print('--')
print(test_ds.tail())

         x      y
8673  8673  17351
8175  8175  16355
4933  4933   9871
504    504   1013
107    107    219
--
         x      y
9963  9963  19931
9969  9969  19943
9972  9972  19949
9982  9982  19969
9995  9995  19995


In [5]:
train_ds.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
x,8000.0,5003.4595,2887.354272,1.0,2493.75,5006.5,7517.25,9999.0
y,8000.0,10011.9190,5774.708543,7.0,4992.50,10018.0,15039.50,20003.0


In [6]:
train_features = train_ds.copy()
test_features = test_ds.copy()

train_labels = train_features.pop('y')
test_labels = test_features.pop('y')

In [7]:
tsr_train_features = tf.convert_to_tensor(train_features)
tsr_train_labels = tf.convert_to_tensor(train_labels)

tsr_test_features = tf.convert_to_tensor(test_features)
tsr_test_labels = tf.convert_to_tensor(test_labels)

2024-02-27 23:45:25.173671: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-27 23:45:25.177288: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-27 23:45:25.180221: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
#normalizer = tf.keras.layers.Normalization(axis=-1)
#normalizer.adapt(np.array(tsr_train_features))

In [9]:
a = np.array(train_features)

a_normalizer = layers.Normalization(input_shape=[1,], axis=None)
a_normalizer.adapt(a)
print ('Done')

2024-02-27 23:45:25.562299: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type CPU is enabled.
2024-02-27 23:45:26.037592: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Done


In [10]:
def build_dnn_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(loss='mean_absolute_error',
        optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

In [11]:
# Normalizer

a_dnn_model = build_dnn_model(a_normalizer)

In [12]:
a_dnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 1)                 3         
 on)                                                             
                                                                 
 dense (Dense)               (None, 64)                128       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4356 (17.02 KB)
Trainable params: 4353 (17.00 KB)
Non-trainable params: 3 (16.00 Byte)
_________________________________________________________________


In [13]:
%%time
a_dnn_model.fit(
    train_features['x'],
    train_labels,
    validation_split=0.2,
    epochs=100)

Epoch 1/100


2024-02-27 23:45:28.154723: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1531cef9a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-27 23:45:28.154755: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 4000, Compute Capability 7.5
2024-02-27 23:45:28.159893: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-27 23:45:28.178417: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-02-27 23:45:28.278229: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


200/200 [==============================] - 2s 4ms/step - loss: 10031.7939 - val_loss: 9715.6396
Epoch 2/100
200/200 [==============================] - 0s 2ms/step - loss: 9648.9648 - val_loss: 8964.2314
Epoch 3/100
200/200 [==============================] - 0s 2ms/step - loss: 8436.2119 - val_loss: 7289.3950
Epoch 4/100
200/200 [==============================] - 0s 2ms/step - loss: 5927.7568 - val_loss: 3936.8840
Epoch 5/100
200/200 [==============================] - 0s 2ms/step - loss: 1736.5490 - val_loss: 492.5401
Epoch 6/100
200/200 [==============================] - 0s 2ms/step - loss: 394.7886 - val_loss: 373.4446
Epoch 7/100
200/200 [==============================] - 0s 2ms/step - loss: 300.8323 - val_loss: 290.0189
Epoch 8/100
200/200 [==============================] - 0s 2ms/step - loss: 232.8318 - val_loss: 226.9003
Epoch 9/100
200/200 [==============================] - 0s 2ms/step - loss: 180.1142 - val_loss: 177.5081
Epoch 10/100
200/200 [==============================] - 0

In [14]:
a1 = [[11000]]
b1 = a_dnn_model.predict(a1)

1/1 [==============================] - 0s 108ms/step


In [15]:
b1

array([[22008.01]], dtype=float32)

In [ ]:
# y_pred is pretty close to 22005, the actual value of the function.